In [ ]:
from ipyaladin import Aladin
from jdaviz import Imviz

uri = "mast:JWST/product/jw02727-o002_t062_nircam_clear-f090w_i2d.fits"

imviz = Imviz()
imviz.load_data(uri, cache=True)
imviz.link_data('wcs')
imviz.show('sidecar:split-bottom')

In [ ]:
imviz.plugins['Orientation'].set_north_up_east_left()

In [ ]:
default_viewer = imviz.app.get_viewer('imviz-0')
default_viewer.toolbar.tools['jdaviz:boxzoom'].activate()
default_viewer.toolbar.active_tool = default_viewer.toolbar.tools['jdaviz:boxzoom']
default_viewer.toolbar.active_tool = default_viewer.toolbar.tools['jdaviz:imagepanzoom']

In [ ]:
def get_image_viewer_fov(default_viewer): 
    reference_data = default_viewer.state.reference_data
    x_center = 0.5 * (default_viewer.state.x_min + default_viewer.state.x_max)
    y_center = 0.5 * (default_viewer.state.y_min + default_viewer.state.y_max)
    center = reference_data.coords.pixel_to_world(x_center, y_center)
    x_span = reference_data.coords.pixel_to_world(
        [default_viewer.state.x_min, default_viewer.state.x_max], 
        [default_viewer.state.y_min, default_viewer.state.y_min]
    )
    y_span = default_viewer.layers[0].layer.data.coords.pixel_to_world(
        [default_viewer.state.x_min, default_viewer.state.x_min], 
        [default_viewer.state.y_min, default_viewer.state.y_max]
    )
    max_fov = max(
        x_span[0].separation(x_span[1]),
        y_span[0].separation(y_span[1])
    )

    return center, max_fov
    

def get_aladinlite_fov(default_viewer):
    coord, max_fov = get_image_viewer_fov(default_viewer)
    return coord.to_string('hmsdms', sep=' '), max_fov

get_aladinlite_fov(default_viewer)

In [ ]:
from glue.viewers.common.viewer import Viewer
from ipywidgets import Layout, VBox
import astropy.units as u

# SkyCoord with center of image
image_center = imviz.app.data_collection[0].coords.pixel_to_world(0, 0)

class AladinViewer(Viewer):
    # items = [
    #     Aladin(target=f"{image_center.ra.deg} {image_center.dec.deg}")
    # ]

    # box_layout = Layout(
    #     display="flex", flex_flow="row", align_items="stretch", width="100%", height="100%"
    # )
    _deprecated_data_menu = None
    _marktags = []
    aladin = Aladin(
        target=f"{image_center.ra.deg} {image_center.dec.deg}", 
        # height=475,
        fov=0.1
    )
    # box = VBox([aladin])
    # figure = box
    figure = aladin
    toolbar = None
    
    @property
    def figure_widget(self):
        return self.figure

In [ ]:
from jdaviz.core.events import SnackbarMessage, NewViewerMessage, ViewerAddedMessage

self = imviz.app
vid = name = 'aladin'
msg = NewViewerMessage(AladinViewer, data=None, sender=imviz.app)

viewer = self._application_handler.new_data_viewer(
    msg.cls, data=msg.data, show=False)
# viewer.figure_widget.layout.height = '100%'

default_viewer.toolbar.active_tool = default_viewer.toolbar.tools['jdaviz:boxzoom']
default_viewer.toolbar.active_tool = default_viewer.toolbar.tools['jdaviz:imagepanzoom']

new_viewer_item = self._create_viewer_item(
    viewer=viewer, vid=vid, name=name, reference=name,
    open_data_menu_if_empty=False
)

new_stack_item = self._create_stack_item(
    container='gl-stack',
    viewers=[new_viewer_item])

# Store the glupyter viewer object so we can access the add and remove
#  data methods in the future
vid = new_viewer_item['id']
self._viewer_store[vid] = viewer

# Add viewer locally
self.state.stack_items.append(new_stack_item)

self.session.application.viewers.append(viewer)

self.hub.broadcast(ViewerAddedMessage(vid, sender=self))


aladin_viewer = imviz.app.get_viewer('aladin')

target, fov = get_image_viewer_fov(default_viewer)
aladin_viewer.figure.target = target

from astropy.time import Time
from echo import delay_callback


def _sync_fov():
    print(f'call _sync_center @ {Time.now().iso}')
    target, fov = get_image_viewer_fov(default_viewer)
    # all_keys = ['fov', 'target']
    #with delay_callback(aladin_viewer.figure):
    aladin_viewer.figure.fov = fov.to_value(u.deg)
    aladin_viewer.figure.target = target
    
default_viewer.state.add_callback('x_min', lambda x_min: _sync_fov())